# QM/MM in Gromos

In [ ]:
# necessary imports are the Gromos_System and convenience functions emin and md
import pygromos
from pygromos.files.simulation_parameters.imd import Imd
from pygromos.files.qmmm.qmmm import QMMM
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.simulations.modules.preset_simulation_modules import emin, md
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL

# for file paths
import os

## Support for QMMM functionality in `GROMOS` input files

This notebook demonstrates support of `PyGromosTools` for QM/MM functionality.

https://github.com/rinikerlab/PyGromosTools/blob/qmmm/examples/example_gromos_qmmm.ipynb (part of the `qmmm` branch and soon to be merged to `release3`)

Author: Felix Pultar


Features include:

* QM/MM blocks in `imd` files
* QM/MM specification files
* Running QM/MM simulations

### Load an `imd` file containing a QMMM block <a class="anchor" id="imd-files"></a>
Simple demonstration of how to handle `.imd` files.

In [ ]:
imd_path = os.path.abspath("example_files/QMMM_files/md.imd")
imd_file = Imd(imd_path)
imd_file.TITLE.content = "Demonstration of a Gromos imd file containing a QMMM block"
imd_file

### Print out different sections of the QMMM block

Print out selected parameters from the `QMMM` block or also the `TITLE` block.

In [ ]:
print(imd_file.QMMM.NTQMMM) # QM/MM toggled on/off
print(imd_file.QMMM.NTQMSW) # which QM/MM engine
print(imd_file.TITLE.content)

### Change a block value and print again

Just change values of the `QMMM` block like with other `PyGromosTools` blocks.

In [ ]:
imd_file.QMMM.NTQMSW = 4 # switch to ORCA as QM software
imd_file.QMMM

## Directly manipulate a QMMM specification file

The QMMM object allows to directly interact with QM/MM specification files. Future releases of `PyGromosTools` will also support generation of `QMMM` files from coordinate files (`.cnf`, `.xyz`, `.pdb`).

In [ ]:
# instantiate the file object
qmmm_file = QMMM("example_files/QMMM_files/menthol-methanol-dmf.qmmm")
print(qmmm_file)
# There will be warnings if more than one QM engine is selected

## Print out and change some blocks in the QMMM specification file

### Title block

The `QMMM` specification file can be handled like any other `GROMOS` file.

In [ ]:
print(qmmm_file.TITLE.content)
qmmm_file.TITLE.content = "Custom file header"
print(qmmm_file.TITLE.content)

### QMZONE block

Print out the `QMZONE` section that defines which atoms will be treated quantum-mechanically.

In [ ]:
# as in other Gromos files, the first bunch of characters are ignored and used to comment, e.g. name of the atom
# second value: index of te position (starting from 1)
# third value: element number according to the PSE
# fourth value: indicate whether bond can be broken or not, default = 0
print(qmmm_file.QMZONE)

### QMUNIT block

Print out the `QMUNIT` block that defines some unit conversions between the MD engine and the QM software.

In [ ]:
# usually, these conversion factors are hard-coded in Gromos; left for historical reasons
# first value: QM length to Gromos length (e.g. Bohr to nm) 
# second value: QM energy to Gromos energy (e.g. Hartree to kJ / mol)
# third value: Gromos charge to QM charge (the same in this case)
# fourth value: QM input units to Gromos input units (e.g. Angstrom to nm)
print(qmmm_file.QMUNIT)

### XTBELEMENTS block

Print and update the `XTBELEMENTS` block

In [ ]:
print(qmmm_file.XTBELEMENTS)
print(qmmm_file.XTBELEMENTS.content)

In [ ]:
# replace element numbers manually with the first ten elements of the PSE
xtbelements_new  = [[str(i) for j in range(1)] for i in range(1,11)]
print(xtbelements_new)
qmmm_file.XTBELEMENTS.content = xtbelements_new

In [ ]:
# show the updated section in the file
qmmm_file.XTBELEMENTS

### A helper function that returns all QM engines specified in the QM/MM specification file

There is also a sanity check in the constructor of `QMMM` to see if you did not accidentally add more than one QM engine.

In [ ]:
print(qmmm_file.get_qm_engines())

### Store your QMMM specification file with all your other simulation files in a `Gromos_System` object

In [ ]:
# that's what we want to simulate
system_name = "menthol-methanol-dmf"

# that's where we want to simulate it at
work_folder = f"example_files/{system_name}"

# create a Gromos_System object from scratch
system = Gromos_System(work_folder, system_name)

# specify prepared topology, configuration, QMMM specification file, and input file
system.top = f"example_files/QMMM_files/{system_name}-all-atom_54a7.top"
system.cnf = f"example_files/QMMM_files/{system_name}-all-atom-init_54a7.cnf"
system.qmmm = qmmm_file
system.imd = imd_file

# clean up
system.rebase_files()

# all your simulation files now live in the work folder
system

In [ ]:
# the imd file has been adapted (force groups, multibath, etc.)
system.imd

## Run QM/MM Simulations

QM/MM calculations are possible with a special in-house build of `GROMOS`.

In [ ]:
# binaries (not yet QM/MM)
gromosPP = None
gromosXX = None

# folders and title
system_name = "menthol-methanol-dmf"
work_folder =  f"example_files/{system_name}"

# files
in_cnf_path  = f"example_files/QMMM_files/menthol-methanol-dmf-all-atom-init_54a7.cnf"
in_top_path  = f"example_files/QMMM_files/menthol-methanol-dmf-all-atom_54a7.top"

# system
system = Gromos_System(
    work_folder, 
    system_name, 
    in_top_path=in_top_path, 
    in_cnf_path=in_cnf_path,
    in_gromosPP_bin_dir=gromosPP,
    in_gromosXX_bin_dir=gromosXX
)

system

In [ ]:
system.cnf.visualize()

In [ ]:
# create a local submission system and specify the number of cores
submit = LOCAL(nomp=8)

# energy minimize the system
minimized_system = emin(system, submission_system=submit)

In [ ]:
# new imd file for equilibration
in_imd_path  = f"example_files/QMMM_files/menthol-methanol-dmf-eq.imd"
minimized_system.imd = in_imd_path
minimized_system.imd.STEP.NSTLIM = 1000
minimized_system.imd.STEP.DT = 0.002 # 2.0 fs
minimized_system

In [ ]:
# equilibrate the system
equilibrated_system = md(minimized_system, step_name="eq", submission_system=submit)
equilibrated_system

In [ ]:
# load your favorite GCC version (required for your special GROMOS QM/MM build)
os.environ["PATH"] += "/opt/gcc-8.2.0/bin" #Change here!
os.environ["LD_LIBRARY_PATH"] = "/opt/gcc-8.2.0/lib:/home/fpultar/opt/gcc-8.2.0/lib64" #Change here!
os.environ

In [ ]:
# new imd file for QM/MM run
in_imd_path  = f"example_files/QMMM_files/md.imd"
equilibrated_system.imd = in_imd_path
equilibrated_system.imd.STEP.NSTLIM = 100
equilibrated_system.imd.STEP.DT = 0.0005 # 0.5 fs
# qmmm specification file
equilibrated_system.qmmm = QMMM(f"example_files/QMMM_files/menthol-methanol-dmf.qmmm")

# now you want to switch to your special build of GROMOS :)
gromosXX = None # path to freshly compiled gromos "build-gcc-8.2.0-release/program"
equilibrated_system.gromosXX = gromosXX

# check if everying is correct
# note that the new .imd file and .qmmm file still live in the old location
# while the .cnf file and .top file result from a previous simulation (equilibration)
equilibrated_system

In [ ]:
# print the relevant QMMM block in the new imd file
equilibrated_system.imd.QMMM

In [ ]:
# go QM/MM!
production_system = md(equilibrated_system, submission_system=submit)

In [ ]:
# visualize the last .cnf - you are done!
production_system.cnf.visualize()